## Method 1



In [506]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

rho_oil = 875.3
rho_air = 1.204
g = 9.80
nu = 1.827e-5
d = 6.0e-3
precision = 19

velocities_file = '/Users/lucaschoi/Documents/GitHub/PHY294-Milikan-Oil-Drop-Experiment/velocities.tsv'
method1_results_file = '/Users/lucaschoi/Documents/GitHub/PHY294-Milikan-Oil-Drop-Experiment/method1.py'

charges = []
charge_uncs = []
V_unc = 0.3

velocities_df = pd.read_csv(velocities_file, sep='\t')



In [507]:
for _, row in velocities_df.iterrows():
    try:
        V_stop = row['stopping_voltage']
        v_d = row['v_fall']
        v_d_unc = row['v_fall_unc']
        
        C1 = ((18 * np.pi * nu) / (rho_oil * g))**(3/2) / d
        q = C1 * v_d ** (3/2) / V_stop

        # Calculate the uncertainty in charge
        partial_q_vd = (3 / 2) * (v_d ** (1/2)) / V_stop
        partial_q_V_stop = -(v_d ** (3/2)) / (V_stop ** 2)
        q_unc = np.sqrt((partial_q_vd * v_d_unc) ** 2 + (partial_q_V_stop * V_unc) ** 2)
        charges.append(q)
        charge_uncs.append(q_unc)
        # print(f'q = {q:.10} \pm {q_unc:.10} C')

    except Exception as e:
        print(f"{e}")
all_e = []

In [508]:

def estimate_elementary_charge(q_list, e_min=1e-19, e_max=2e-19, num_steps=1000000):
    """
    Estimate the elementary charge by trying possible divisors of q_list.
    
    Args:
        q_list (array-like): Measured charges in Coulombs
        e_min, e_max: Range to scan for candidate e
        num_steps: Number of candidate e values to test
    
    Returns:
        best_e: Estimated elementary charge
        scores: List of scores for all candidate e values
    """
    global all_e

    q_array = np.array(q_list)
    candidate_es = np.linspace(e_min, e_max, num_steps)
    best_score = float('inf')
    best_e = None
    scores = []

    for e in candidate_es:

        # Divide all q values by e and check how close to nearest integer
        multiples = q_array / e
        residuals = np.abs(multiples - np.round(multiples))
        score = np.mean(residuals) 
        scores.append(score)
        all_e.append((e, score))
        
        if score < best_score:
            best_score = score
            best_e = e

    return best_e, scores, candidate_es

In [509]:
print(min(charges), max(charges))

4.535594081915622e-18 7.427327701625683e-17


In [510]:

e_max = 1e-18
e_min = 1e-20
estimated_e = []
probe_min = e_min + 1
probe_max = e_max
all_e = []

while probe_min > e_min:
    probe_min = probe_max / 10
    probe_max = probe_max
    print(f"Testing range: e_min = {probe_min:.3e}, e_max = {probe_max:.3e}")
    
    best_e, scores, candidate_es = estimate_elementary_charge(
        charges,
        e_min=probe_min,
        e_max=probe_max,
        num_steps=1000000
    )
    
    estimated_e.append((probe_min, probe_max, best_e, scores))
    probe_max /= 10
    


Testing range: e_min = 1.000e-19, e_max = 1.000e-18
Testing range: e_min = 1.000e-20, e_max = 1.000e-19
Testing range: e_min = 1.000e-21, e_max = 1.000e-20


In [511]:
# sort the all_e's by the best score
all_e.sort(key=lambda x: x[1])

for i in range(100):
    print(f"Best e = {all_e[i][0]:.3e} C with score {all_e[i][1]}")

# average of best 10 ** n
for i in range(5):

    scale = 10 ** i
    if scale > len(all_e):
        break
    print(f"Average of best {scale} = {np.mean([all_e[i][0] for i in range(scale)]):.3e} C")

print(max(all_e[:100], key=lambda x: x[0]))


Best e = 1.274e-21 C with score 0.15588971622772826
Best e = 3.669e-21 C with score 0.16180929436920594
Best e = 3.669e-21 C with score 0.1626178291283832
Best e = 3.669e-21 C with score 0.1634054436419997
Best e = 3.669e-21 C with score 0.16485928307268777
Best e = 3.669e-21 C with score 0.1648607242113806
Best e = 3.669e-21 C with score 0.16728264977526872
Best e = 3.669e-21 C with score 0.16839604912471864
Best e = 1.519e-19 C with score 0.1702050708024207
Best e = 1.519e-19 C with score 0.17020670198164217
Best e = 1.519e-19 C with score 0.1702101076754816
Best e = 1.519e-19 C with score 0.1702151396225359
Best e = 1.519e-19 C with score 0.17022507137287757
Best e = 1.519e-19 C with score 0.17022520856195095
Best e = 1.519e-19 C with score 0.17023527762066698
Best e = 1.519e-19 C with score 0.17024032490783209
Best e = 1.519e-19 C with score 0.17024534679868814
Best e = 1.519e-19 C with score 0.17025541609601413
Best e = 1.519e-19 C with score 0.17025557826206286
Best e = 1.519e-19